In [1]:
import os

In [2]:
!pwd

/home/jovyan/notebooks/analysis/experiment_4


In [3]:
import sys
sys.path.append('/home/jovyan/')

In [4]:
from experiments.reporting.report_repository import ReportRepository
from experiments.slack_client import SlackNotifier
from neat.neat_logger import get_neat_logger
from neat.population_engine import EvolutionEngine
from neat.reporting.reports_pyneat import EvolutionReport
from neat.utils import timeit
import math
import random
from itertools import count
import numpy as np

from experiments.logger import logger
from experiments.slack_client import Notifier
from experiments.object_repository.object_repository import ObjectRepository
from experiments.reporting.report_repository import ReportPathFactory
from neat.configuration import get_configuration
from neat.evaluation.evaluation_engine import EvaluationStochasticEngine
from neat.evolution_operators.crossover import Crossover
from neat.evolution_operators.mutation import Mutation
from neat.genome import Genome
from neat.reporting.reports_pyneat import EvolutionReport
from neat.species import SpeciationEngine
from neat.stagnation import Stagnation
from neat.population_engine import PopulationEngine
from neat.utils import timeit
import neat
import tests
from config_files.configuration_utils import create_configuration
from neat.configuration import get_configuration, set_configuration
from neat.evaluation.evaluation_engine import get_dataset
from neat.evaluation.evaluate_simple import evaluate_genome, calculate_prediction_distribution
from neat.genome import Genome
from neat.evolution_operators.backprop_mutation import BackPropMutation
import os
from neat.loss.vi_loss import get_loss
import torch
import pandas as pd

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_curve

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from neat.analysis.experiment_data import ExperimentDataNE, ExperimentDataNAS
from neat.analysis.uncertainty.predictive_distribution import PredictionDistributionEstimator
from neat.analysis.plotting.plot_losses import plot_loss_distribution
from neat.analysis.plotting.plot_classification_metrics import plot_metrics_by_quantile_several_executions
from neat.analysis.experiment_predictive_distributions import ExecutionsPredictionDistributions

In [7]:
LOGS_PATH = f'{os.getcwd()}/'
logger = get_neat_logger(path=LOGS_PATH)

In [10]:
config_versions = {'NE': {'project': 'neuro-evolution',
                         'algorithm_version': 'bayes-neat'},
                  'NAS': {'project': 'nas',
                         'algorithm_version': 'nas'}
                  }
datasets = ['iris', 'wine', 'breast_cancer', 'mnist_downsampled']
# datasets = [ 'mnist_downsampled']
# dataset = 'iris'

In [11]:
# df_versions = pd.DataFrame([['Bayesian-NEAT', 'NE', True, 'bayesian_neat_ft_21_iris'],
#                            ['NEAT', 'NE', False, 'neat_ft_21_iris'],
#                            ['Bayesian-MLP', 'NAS', True, 'bayesian_nas_final'],
# #                            ['NAS', False, 'standard_nas_final_with_reps']
#                            ['MLP', 'NAS', False, 'standard_nas_final']
#                            ], columns=['method', 'search', 'is_bayesian', 'correlation_id'])
versions_map = {}
for dataset in datasets:
    df_versions = pd.read_csv(f'./versions/{dataset}.csv')
    print(f'DATASET: {dataset}')
    print(df_versions)
    versions_map[dataset] = df_versions

DATASET: iris
          method search  is_bayesian            correlation_id
0  Bayesian-NEAT     NE         True  bayesian_neat_ft_21_iris
1           NEAT     NE        False           neat_ft_21_iris
2   Bayesian-MLP    NAS         True        bayesian_nas_final
3            MLP    NAS        False        standard_nas_final
DATASET: wine
          method search  is_bayesian                   correlation_id
0  Bayesian-NEAT     NE         True  bayesian_neat_final_wine_lenovo
1           NEAT     NE        False  standard_neat_final_wine_lenovo
2   Bayesian-MLP    NAS         True     bayesian_nas_final_v1_lenovo
3            MLP    NAS        False     standard_nas_final_v1_lenovo
DATASET: breast_cancer
          method search  is_bayesian                     correlation_id
0  Bayesian-NEAT     NE         True  bayesian_neat_ft_20_breast_cancer
1           NEAT     NE        False           neat_ft_18_breast_cancer
2   Bayesian-MLP    NAS         True                 bayesian_nas_fi

In [11]:
# df_versions.to_csv(f'./versions/{dataset}.csv', index=False)

In [13]:
chunks_ne = []
chunks_nas = []
for dataset, df_versions in versions_map.items():
    print(dataset)
    for index, row in df_versions.iterrows():
        method = row.method
        version = row.search
        is_bayesian = row.is_bayesian
        correlation_id = row.correlation_id
        print(method, version, is_bayesian, correlation_id)
        if isinstance(correlation_id, float):
            continue
            
        n_samples = 1000 if is_bayesian else 1
        project = config_versions[version]['project']
        algorithm_version = config_versions[version]['algorithm_version']
        print(project, algorithm_version, n_samples)
        if version == 'NE':
            experiment_data = ExperimentDataNE(correlation_ids=[correlation_id], 
                                               dataset_name=dataset, 
                                               n_samples=n_samples,
                                               project = project, 
                                               algorithm_version = algorithm_version,
                                               keep_top=1.0, 
                                               filter_checkpoint_finish=True)\
                                .process_data()
            chunk_data_ne = experiment_data.get_experiment_data()
            # stay only with the best
            chunk_data_ne = chunk_data_ne.loc[chunk_data_ne['absolute_best'] == True]
            chunk_data_ne['version'] = version
            chunk_data_ne['is_bayesian'] = is_bayesian
            chunk_data_ne['method'] = method
            chunk_data_ne['dataset'] = dataset
            chunks_ne.append(chunk_data_ne)
        elif version == 'NAS':
            experiment_data_nas = ExperimentDataNAS(correlation_ids=[correlation_id], dataset_name=dataset, n_samples=n_samples,
                                                    project=config_versions[version]['project'], 
                                                    algorithm_version=config_versions[version]['algorithm_version'],
                                                    keep_top=1.0, filter_checkpoint_finish=True)
            experiment_data_nas.process_data()
            chunk_data_nas = experiment_data_nas.experiment_data
            chunk_data_nas['version'] = version
            chunk_data_nas['is_bayesian'] = is_bayesian
            chunk_data_nas['method'] = method
            chunk_data_nas['dataset'] = dataset
            chunks_nas.append(chunk_data_nas)
        else:
            raise ValueError()
data_ne = pd.concat(chunks_ne, sort=False)
data_nas = pd.concat(chunks_nas, sort=False)
# bayesian_neat_ft_final_v1_spambase

iris
Bayesian-NEAT NE True bayesian_neat_ft_21_iris
neuro-evolution bayes-neat 1000
###########
CORRELATION ID: bayesian_neat_ft_21_iris
45 executions
Train percentage: 0.75
Random state: 53
Sum Train: -11.794641494750977
Label Noise: 0.3
Sum of 10 first labels: 11
[1 2 2 0 1 2 2 1 0 0]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 5
Sum Train: 0.2868227958679199
Label Noise: 0.3
Sum of 10 first labels: 11
[0 2 2 1 0 0 2 2 0 2]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 48
Sum Train: -9.0440034866333
Label Noise: 0.4
Sum of 10 first labels: 10
[0 2 1 1 2 2 0 2 0 0]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 37
Sum Train: -9.825719833374023
Label Noise: 0.2
Sum of 10 first labels: 8
[2 0 0 2 0 2 0 1 1 0]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 98
Sum Train: -12.14836597442627
Label Noise: 0.7
Sum of 10 first labels: 4
[0 0 2 0 1 0 1 0 0 0]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 3

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train percentage: 0.75
Random state: 3
Sum Train: 11.042451858520508
Label Noise: 0.5
Sum of 10 first labels: 7
[0 2 2 0 0 1 1 1 0 0]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 42
Sum Train: -0.8510899543762207
Label Noise: 0.7
Sum of 10 first labels: 6
[1 0 0 1 0 1 1 0 1 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 57
Sum Train: -5.5301971435546875
Label Noise: 0.5
Sum of 10 first labels: 9
[0 1 1 2 2 0 1 0 1 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 71
Sum Train: -7.959840774536133
Label Noise: 0.8
Sum of 10 first labels: 10
[1 1 1 1 1 0 2 2 0 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 62
Sum Train: -4.483011245727539
Label Noise: 0.8
Sum of 10 first labels: 12
[2 0 2 1 0 2 2 0 2 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 41
Sum Train: 4.1221747398376465
Label Noise: 0.1
Sum of 10 first labels: 12
[2 1 1 1 0 2 2 0 2 1]
Training: 112. Testing: 38
Train percentage: 0.75
R

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train percentage: 0.75
Random state: 88
Sum Train: -6.814027786254883
Label Noise: 0.0
Sum of 10 first labels: 12
[0 2 2 2 2 0 1 2 1 0]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 73
Sum Train: 0.6495952606201172
Label Noise: 0.1
Sum of 10 first labels: 11
[1 0 2 1 1 2 1 0 2 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 16
Sum Train: 12.39934253692627
Label Noise: 0.1
Sum of 10 first labels: 12
[2 0 2 2 0 1 1 2 1 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 6
Sum Train: 8.02547836303711
Label Noise: 0.2
Sum of 10 first labels: 8
[1 0 0 2 2 1 0 0 2 0]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 26
Sum Train: 3.800556182861328
Label Noise: 0.5
Sum of 10 first labels: 8
[0 0 0 2 0 2 2 0 0 2]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 56
Sum Train: 7.987251281738281
Label Noise: 0.4
Sum of 10 first labels: 14
[2 0 1 2 2 1 1 2 2 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random 

...

Train percentage: 0.75
Random state: 22
Sum Train: -6406.79931640625
Label Noise: 0.95
Sum of 10 first labels: 33
[9 0 7 1 5 1 0 3 7 0]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 22
Sum Train: -6406.79931640625
Label Noise: 0.95
Sum of 10 first labels: 33
[9 0 7 1 5 1 0 3 7 0]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 7
Sum Train: -6844.130859375
Label Noise: 0.1
Sum of 10 first labels: 50
[8 1 3 9 1 4 8 7 2 7]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 85
Sum Train: 225.28805541992188
Label Noise: 0.95
Sum of 10 first labels: 21
[0 2 3 3 1 4 0 0 4 4]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 75
Sum Train: -6569.7353515625
Label Noise: 0.95
Sum of 10 first labels: 64
[7 9 8 6 6 5 6 8 0 9]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 75
Sum Train: -6569.7353515625
Label Noise: 0.95
Sum of 10 first labels: 64
[7 9 8 6 6 5 6 8 0 9]
Training: 3750. Testing: 1250
Train per

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train percentage: 0.75
Random state: 33
Sum Train: -6713.39599609375
Label Noise: 0.5
Sum of 10 first labels: 63
[5 1 8 9 3 9 9 5 6 8]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 78
Sum Train: -7216.935546875
Label Noise: 0.7
Sum of 10 first labels: 42
[1 4 5 4 7 0 7 9 4 1]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 47
Sum Train: -7140.75732421875
Label Noise: 0.5
Sum of 10 first labels: 50
[6 7 1 6 0 9 5 7 8 1]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 47
Sum Train: -7140.75732421875
Label Noise: 0.3
Sum of 10 first labels: 63
[7 7 1 7 6 9 7 7 4 8]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 36
Sum Train: -6951.80908203125
Label Noise: 0.4
Sum of 10 first labels: 35
[0 2 9 5 0 2 7 5 2 3]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 72
Sum Train: -6519.06494140625
Label Noise: 0.2
Sum of 10 first labels: 36
[2 9 3 0 2 6 0 2 9 3]
Training: 3750. Testing: 1250
Train percen

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

NEAT NE False neat_ft_final_v1_label_noise_mnist_downsampled
neuro-evolution bayes-neat 1
###########
CORRELATION ID: neat_ft_final_v1_label_noise_mnist_downsampled
45 executions
Train percentage: 0.75
Random state: 84
Sum Train: -6789.4111328125
Label Noise: 0.1
Sum of 10 first labels: 33
[6 0 0 6 1 5 0 8 6 1]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 84
Sum Train: -6789.4111328125
Label Noise: 0.1
Sum of 10 first labels: 33
[6 0 0 6 1 5 0 8 6 1]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 11
Sum Train: -7654.67138671875
Label Noise: 0.5
Sum of 10 first labels: 47
[9 0 4 3 9 3 5 9 1 4]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 11
Sum Train: -7654.67138671875
Label Noise: 0.5
Sum of 10 first labels: 47
[9 0 4 3 9 3 5 9 1 4]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 86
Sum Train: -6473.2705078125
Label Noise: 0.6
Sum of 10 first labels: 41
[2 5 0 7 0 1 7 9 7 3]
Training: 3750. Testing: 1250

In [14]:
display(data_ne.head(1))
display(data_nas.head(1))

,correlation_id,execution_id,train_percentage,noise,label_noise,is_bayesian,beta,loss_training,loss_testing,duration,...,mean_genome_std,accuracy,precision,recall,f1,ece,absolute_best,version,method,dataset
0,bayesian_neat_ft_21_iris,068b18e4-a7fb-4df5-9441-f6f543069c2f,0.75,0.0,0.3,True,0.0001,0.922851,0.863731,0:15:02.814875,...,0.772447,92.105263,0.923347,0.921053,0.921358,0.145418,True,NE,Bayesian-NEAT,iris


,correlation_id,execution_id,train_percentage,noise,label_noise,is_bayesian,beta,duration,end_condition,n_parameters,...,n_connections,n_layers,f1,recall,precision,accuracy,ece,version,method,dataset
0,bayesian_nas_final,04c937ef-f1f6-433c-a501-de48fd5453cf,0.75,0.0,0.0,True,0.000001,0:06:23.322305,normal,326,...,140,2,0.947169,0.947368,0.955466,94.736842,0.045219,NAS,Bayesian-MLP,iris


In [15]:
from scipy import stats
def filter_data(data, filter_noisy=False, z_score=3):
#     data = data.loc[data['f1'] >= filter_threshold]
    chunks = []
    for _, df in data.groupby(['method', 'label_noise']):
#         df[(np.abs(stats.zscore(df['f1'])) < 3).all(axis=1)]
        df = df.loc[np.abs(stats.zscore(df['f1'])) < z_score]
        chunks.append(df)
    data = pd.concat(chunks)
    if filter_noisy:
        data = data.loc[(data['noise'] == 0) &
               (data['label_noise'] == 0)]
    
    
    return data

Filter not noisy executions

In [16]:
display(data_ne.shape)
display(data_nas.shape)

(473, 25)

(550, 21)

In [17]:
data_ne = data_ne.loc[(data_ne['noise'] != 0) |
                     (data_ne['label_noise'] != 0)]
data_nas = data_nas.loc[(data_nas['noise'] != 0) |
                     (data_nas['label_noise'] != 0)]

In [18]:
display(data_ne.shape)
display(data_nas.shape)

(427, 25)

(480, 21)

In [19]:
version = 1
for dataset, data_dataset in data_ne.groupby('dataset'):
    data_dataset.to_csv(f'data/data_experiment_4_ne_{dataset}_version_{version}.csv', index=False)

for dataset, data_dataset in data_nas.groupby('dataset'):
    data_dataset.to_csv(f'data/data_experiment_4_nas_{dataset}_version_{version}.csv', index=False)
